<a href="https://colab.research.google.com/github/marcosrodsa/projeto_imersao_alura/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando Dependências

In [1]:
!pip install -q -U google-generativeai

In [2]:
!pip install -q -U geopy

In [3]:
!pip install -q -U google-api-python-client

In [4]:
!pip install -q -U google-search-results

In [5]:
!pip install newspaper3k -q -U

In [6]:
!pip install unstructured -q -U

Inicando Funções

In [9]:
import google.generativeai as genai
from google.colab import userdata

from IPython.display import clear_output
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut
from googleapiclient.discovery import build

import newspaper
import nltk
import requests
import json
import os
import time
from unstructured.partition.html import partition_html  # Importando a função
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from googleapiclient.discovery import build
from newspaper import Article, ArticleException  # Import ArticleException
import textwrap
import re
from googleapiclient.errors import HttpError

from IPython.display import display
from IPython.display import Markdown

from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
SEARCH_ENGINE_ID = userdata.get('SEARCH_ENGINE_ID')

genai.configure(api_key=google_api_key)

generation_config = {
	'candidate_count': 1,
	'temperature': 0.5
}

safety_settings = {
	"HARASSMENT": "BLOCK_NONE",
	"HATE": "BLOCK_NONE",
	"SEXUAL": "BLOCK_NONE",
	"DANGEROUS": "BLOCK_NONE"
}

model = genai.GenerativeModel(model_name="gemini-1.0-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

chat = model.start_chat(history=[])

def to_markdown(text, role):
    """Converts text to Markdown with styling based on the role."""
    if role == 'user':
        return Markdown(f'**{role}**: {text}')  # Bold user text
    else:
        return Markdown(textwrap.indent(text, '> ', lambda line: True))  # Indent assistant text

def resumir_noticia_com_gemini(url):
    """Resume a notícia com o Gemini, verificando a conexão com a internet e pulando silenciosamente em caso de erro."""
    try:
        requests.head(url, timeout=5)  # Testa a conexão com a URL
        prompt = f"Resuma a notícia e coloque o link da notícia e a data da publicação se tiver:\n\n{url}"
        resposta = chat.send_message(prompt)
        return resposta.text
    except requests.ConnectionError:
        return None  # Retorna None em caso de erro de conexão

def prever_areas_de_risco():
    #clear_output()
    print("**Prever Possíveis Áreas de Risco**")
    cidade = input("Digite o nome da sua cidade: ")
    historico_enchentes = obter_historico_enchentes(cidade)  # Função para obter dados históricos
    previsao_tempo = obter_previsao_tempo(cidade)  # Função para obter previsão do tempo

    prompt = f"Com base no histórico de enchentes em {cidade} ({historico_enchentes}) e na previsão do tempo ({previsao_tempo}), quais as áreas com maior risco de inundação?"
    resposta = chat.send_message(prompt)

    if resposta:
        print("Áreas com maior risco de inundação:")
        print(resposta.text)
    else:
        print("Não foi possível prever as áreas de risco.")

def obter_historico_enchentes(cidade):
    """Obtém informações sobre o histórico de enchentes em uma cidade."""

    query = f"histórico de enchentes em {cidade}, Rio Grande do Sul"
    resultados = buscar_informacoes_google_cse(query, GOOGLE_API_KEY, SEARCH_ENGINE_ID)

    historico = ""
    for resultado in resultados:
        resumo = resumir_noticia_com_gemini(resultado['link'])
        if resumo:
            historico += resumo + "\n\n"

    if not historico:
        historico = f"Não foram encontradas informações sobre o histórico de enchentes em {cidade}."

    return historico


def obter_previsao_tempo(cidade):
    """Obtém a previsão do tempo para uma cidade."""

    query = f"previsão do tempo em {cidade}, Rio Grande do Sul"
    resultados = buscar_informacoes_google_cse(query, GOOGLE_API_KEY, SEARCH_ENGINE_ID)

    previsao = ""
    for resultado in resultados:
        resumo = resumir_noticia_com_gemini(resultado['link'])
        if resumo:
            previsao += resumo + "\n\n"

    if not previsao:
        previsao = f"Não foi possível obter a previsão do tempo para {cidade}."

    return previsao

def responder_pergunta_sobre_noticia(noticias, pergunta):
    """Responde a uma pergunta sobre as notícias fornecidas usando o Gemini."""
    try:
        #todos_os_textos = "\n\n".join([n['texto'] for n in noticias if 'texto' in n])  # Verifica se a chave 'texto' existe
        prompt = f"Responda à pergunta com base nas seguintes notícias:\n\n{noticias}\n\nPergunta: {pergunta}"

        # Gemini Pro API call (replace with your actual implementation)
        resposta = chat.send_message(prompt)

        if resposta:
            return resposta.text
        else:
            return "Gemini Pro não encontrou informações suficientes para responder à pergunta."

    except Exception as e:
        print(f"Erro ao gerar resposta com o Gemini Pro: {e}")
        return "Ocorreu um erro ao processar sua pergunta."

def gerar_alerta_enchente(cidade):
    """Gera um alerta personalizado sobre enchentes em uma cidade usando o Gemini."""
    prompt = f"Gere um alerta sobre enchentes em {cidade}, Rio Grande do Sul, com informações sobre áreas afetadas, recomendações de segurança e locais de doação."
    resposta = chat.send_message(prompt)
    return resposta.text

from datetime import datetime

def buscar_informacoes_google_cse(query, api_key, search_engine_id, num_results=10, start_index=1):
    """Busca informações sobre enchentes e alagamentos em 2024, excluindo resultados do YouTube."""
    try:
        service = build("customsearch", "v1", developerKey=api_key)

        all_results = []
        while len(all_results) < num_results:
            result = service.cse().list(
                q=query, cx=search_engine_id, num=min(10, num_results - len(all_results)), start=start_index,
                sort='date:r:20240101:20241231'
            ).execute()
            items = result.get("items", [])
            if not items:
                break

            for item in items:
                # Verificar se o link não é do YouTube
                if not item["link"].startswith("https://www.youtube.com/"):
                    # Filtrar por relevância (palavras-chave relacionadas a enchentes no RS)
                    if any(keyword in item["title"].lower() or keyword in item["snippet"].lower()
                           for keyword in ["enchente", "alagamento", "inundação", "rio grande do sul", "rs"]):
                        all_results.append({
                            "titulo": item["title"],
                            "link": item["link"],
                            "snippet": item["snippet"],
                        })

            start_index += 10
            time.sleep(1)

        return all_results

    except HttpError as e:
        print(f"Erro na solicitação HTTP: {e}")
    except Exception as e:
        print(f"Erro ao buscar informações: {e}")
    return []


def valida_localizacao(cidade, geolocator=None):
  if geolocator is None:
      geolocator = Nominatim(user_agent="verificador_cidade_br")

  try:
      localizacao = geolocator.geocode(cidade, country_codes="BR", language="pt-BR")
      #print(localizacao)
      if localizacao:
          # Verifica se o tipo de lugar é 'city' ou se o nome da cidade está no endereço
          if (
              localizacao.raw.get("place_type") == "city"
              or cidade.lower() in localizacao.address.lower()
          ):
              return True

      return False

  except (GeocoderUnavailable, GeocoderTimedOut) as e:
      print(f"Erro ao acessar o serviço de geocodificação: {e}")
      return False
  except KeyError:
      return False

  try:
      # Tenta encontrar a cidade utilizando o Geopy
      localizacao = geolocator.geocode(cidade)
      return True
  except Exception:
      return False

def informacoes_enchentes():
    #clear_output()
    print("**Últimas Informações sobre as Enchentes no RS**")

    query = f"Enchentes no Rio Grande do Sul 2024"
    resultados = buscar_informacoes_google_cse(query, GOOGLE_API_KEY, SEARCH_ENGINE_ID)

    if resultados:
        noticias_formatadas = []
        todos_os_resumos = []  # Lista para armazenar todos os resumos/snippets

        for resultado in resultados:
            #print("\nResultados da busca: results:"+f"{resultado['titulo']} ({resultado['link']}):\n{resultado['snippet']}\n")

            resumo = resumir_noticia_com_gemini(resultado['link'])

            if resumo:
                noticia_formatada = to_markdown(resumo, '')
                todos_os_resumos.append(resumo)
            else:
                noticia_formatada = to_markdown(f"{resultado['titulo']} ({resultado['link']}):\n{resultado['snippet']}\n", '')
                todos_os_resumos.append(resultado['snippet'])

            noticias_formatadas.append(noticia_formatada)
            display(noticia_formatada)
            print('------------')

        # Loop de perguntas após exibir todos os resumos
        while True:
          chat = model.start_chat(history=[])
          pergunta = input("Deseja fazer alguma pergunta sobre as notícias exibidas? (s/n): ")
          if pergunta.lower() == 's':
              pergunta_usuario = input("Digite sua pergunta sobre as notícias: ")  # Input para a pergunta do usuário
              todos_os_textos = "\n\n".join(todos_os_resumos)  # Join the strings directly

              resposta = responder_pergunta_sobre_noticia(todos_os_textos, pergunta_usuario)  # Passar a pergunta do usuário

              if resposta:
                  print(f"Resposta: {resposta}\n")
              else:
                  print("Não foi possível responder à pergunta.\n")
          elif pergunta.lower() == 'n':
              break
          else:
              print("Opção inválida. Digite 's' para sim ou 'n' para não.")

    else:
        print("Nenhum resultado encontrado.")

def lugares_doacoes():
    #clear_output()
    print("**Lugares para Doação**")
    input_cidade = input("Digite um local para doação específico: ").strip().lower()

    if not input_cidade:
        print("Por favor, digite um local para doação.")
        return

    if valida_localizacao(input_cidade):
        query = f"Pontos de doações em {input_cidade} Rio Grande do Sul"
        resultados = buscar_informacoes_google_cse(query, GOOGLE_API_KEY, SEARCH_ENGINE_ID)

        if resultados:
            print("\nResultados da busca:")
            todos_os_resumos = []  # Lista para armazenar resumos/snippets

            for resultado in resultados:
                resumo = resumir_noticia_com_gemini(resultado['link'])

                if resumo:
                    noticia_formatada = to_markdown(resumo, '')
                    todos_os_resumos.append(resumo)
                else:
                    noticia_formatada = to_markdown(f"{resultado['titulo']} ({resultado['link']}):\n{resultado['snippet']}\n", '')
                    todos_os_resumos.append(resultado['snippet'])

                display(noticia_formatada)
                print('------------')

            # Loop de perguntas após exibir todos os resumos
            while True:
                chat = model.start_chat(history=[])  # Reinicia o histórico do chat para cada pergunta
                pergunta = input("Deseja fazer alguma pergunta sobre os locais de doação exibidos? (s/n): ")
                if pergunta.lower() == 's':
                    pergunta_usuario = input("Digite sua pergunta sobre os locais de doação: ")
                    todos_os_textos = "\n\n".join(todos_os_resumos)
                    resposta = responder_pergunta_sobre_noticia(todos_os_textos, pergunta_usuario)

                    if resposta:
                        print(f"Resposta: {resposta}\n")
                    else:
                        print("Não foi possível responder à pergunta.\n")
                elif pergunta.lower() == 'n':
                    break
                else:
                    print("Opção inválida. Digite 's' para sim ou 'n' para não.")
        else:
            print("Nenhum resultado encontrado.")
    else:
        print(f"A cidade '{input_cidade}' não foi encontrada no Brasil ou não é válida.")


def itens_doacoes():
    #clear_output()
    print("**Ítens para Doação**")
    input_cidade = input("Digite um local para doação específico: ").strip().lower()

    if not input_cidade:
        print("Por favor, digite um local para doação.")
        return

    if valida_localizacao(input_cidade):
        query = f"Quais Ítens para doações em {input_cidade} Rio Grande do Sul"
        resultados = buscar_informacoes_google_cse(query, GOOGLE_API_KEY, SEARCH_ENGINE_ID)

        if resultados:
            print("\nResultados da busca:")
            todos_os_resumos = []  # Lista para armazenar resumos/snippets

            for resultado in resultados:
                resumo = resumir_noticia_com_gemini(resultado['link'])

                if resumo:
                    noticia_formatada = to_markdown(resumo, '')
                    todos_os_resumos.append(resumo)
                else:
                    noticia_formatada = to_markdown(f"{resultado['titulo']} ({resultado['link']}):\n{resultado['snippet']}\n", '')
                    todos_os_resumos.append(resultado['snippet'])

                display(noticia_formatada)
                print('------------')

            # Loop de perguntas após exibir todos os resumos
            while True:
                chat = model.start_chat(history=[])  # Reinicia o histórico do chat para cada pergunta
                pergunta = input("Deseja fazer alguma pergunta sobre os ítens para doação exibidos? (s/n): ")
                if pergunta.lower() == 's':
                    pergunta_usuario = input("Digite sua pergunta sobre os ítens para doação: ")
                    todos_os_textos = "\n\n".join(todos_os_resumos)
                    resposta = responder_pergunta_sobre_noticia(todos_os_textos, pergunta_usuario)

                    if resposta:
                        print(f"Resposta: {resposta}\n")
                    else:
                        print("Não foi possível responder à pergunta.\n")
                elif pergunta.lower() == 'n':
                    break
                else:
                    print("Opção inválida. Digite 's' para sim ou 'n' para não.")
        else:
            print("Nenhum resultado encontrado.")
    else:
        print(f"A cidade '{input_cidade}' não foi encontrada no Brasil ou não é válida.")

def informacoes_defesa_civil():
    #clear_output()
    print("**Informações e Alertas da Defesa Civil do RS**")

    query = "Avisos defesa civil enchentes Rio Grande do Sul enchentes"
    resultados = buscar_informacoes_google_cse(query, GOOGLE_API_KEY, SEARCH_ENGINE_ID)

    if resultados:
        print("\nResultados da busca:")
        todos_os_resumos = []  # Lista para armazenar resumos/snippets

        for resultado in resultados:
            resumo = resumir_noticia_com_gemini(resultado['link'])

            if resumo:
                noticia_formatada = to_markdown(resumo, '')
                todos_os_resumos.append(resumo)
            else:
                noticia_formatada = to_markdown(f"{resultado['titulo']} ({resultado['link']}):\n{resultado['snippet']}\n", '')
                todos_os_resumos.append(resultado['snippet'])

            display(noticia_formatada)
            print('------------')

        # Loop de perguntas após exibir todos os resumos
        while True:
            chat = model.start_chat(history=[])  # Reinicia o histórico do chat para cada pergunta
            pergunta = input("Deseja fazer alguma pergunta sobre os Alertas da Defesa Civil (s/n): ")
            if pergunta.lower() == 's':
                pergunta_usuario = input("Digite sua pergunta sobre os Alertas da Defesa Civil: ")
                todos_os_textos = "\n\n".join(todos_os_resumos)
                resposta = responder_pergunta_sobre_noticia(todos_os_textos, pergunta_usuario)

                if resposta:
                    print(f"Resposta: {resposta}\n")
                else:
                    print("Não foi possível responder à pergunta.\n")
            elif pergunta.lower() == 'n':
                break
            else:
                print("Opção inválida. Digite 's' para sim ou 'n' para não.")
    else:
        print("Nenhum resultado encontrado.")


def menu_principal():
  """Exibe o menu principal e interage com o usuário."""
  while True:
    print("\nMenu Principal:")
    print("1. Últimas Informações sobre as Enchentes no RS")
    print("2. Lugares para Doações")
    print("3. Ítens Necessários para Doações")
    print("4. Informações Urgentes da Defesa Civil")
    print("5. Prever Áreas de Risco")
    print("6. Sair")
    opcao = input("Digite sua opção: ")
    if opcao == "1":
      informacoes_enchentes()
    elif opcao == "2":
      lugares_doacoes()
    elif opcao == "3":
      itens_doacoes()
    elif opcao == "4":
      informacoes_defesa_civil()
    elif opcao == "5":
      prever_areas_de_risco()
    elif opcao == "6":
      break
    else:
      print("Opção inválida. Tente novamente.")

# Executar o menu principal
menu_principal()


Menu Principal:
1. Últimas Informações sobre as Enchentes no RS
2. Lugares para Doações
3. Ítens Necessários para Doações
4. Informações Urgentes da Defesa Civil
5. Prever Áreas de Risco
6. Sair
Digite sua opção: 1
**Últimas Informações sobre as Enchentes no RS**


> **Resumo:**
> 
> Em 2024, o estado do Rio Grande do Sul, no Brasil, sofreu graves inundações devido a fortes chuvas. As inundações afetaram mais de 200 municípios, desalojando milhares de pessoas e causando danos significativos a infraestrutura e à agricultura.
> 
> **Link da Notícia:**
> 
> https://en.wikipedia.org/wiki/2024_Rio_Grande_do_Sul_floods
> 
> **Data da Publicação:**
> 
> Não especificada na notícia

------------


> **Resumo:**
> 
> A BBC Brasil publicou uma notícia em 16 de fevereiro de 2023 sobre a decisão do Supremo Tribunal Federal (STF) de derrubar uma liminar que suspendia a nomeação de André Mendonça para o cargo de ministro do STF.
> 
> A liminar havia sido concedida pelo ministro Kassio Nunes Marques em dezembro de 2022, sob o argumento de que a nomeação de Mendonça não havia seguido o rito constitucional adequado. No entanto, o plenário do STF, por maioria de 8 votos a 2, decidiu derrubar a liminar, permitindo que Mendonça tomasse posse como ministro.
> 
> **Link da Notícia:**
> 
> https://www.bbc.com/portuguese/articles/ck5kp34nr1do
> 
> **Data da Publicação:**
> 
> 16 de fevereiro de 2023

------------


...

------------


> **Resumo:**
> 
> Em 10 de maio de 2023, a CNN Brasil publicou uma notícia informando que a Defesa Civil do Rio Grande do Sul emitiu um alerta sobre o risco de inundação do Rio Taquari.
> 
> O alerta foi emitido devido às fortes chuvas que atingem a região, que elevaram o nível do rio. A Defesa Civil recomenda que a população das áreas ribeirinhas fique em alerta e tome medidas de segurança, como retirar móveis e pertences de áreas que possam ser inundadas.
> 
> A notícia destaca que o Rio Taquari é um dos principais rios do Rio Grande do Sul e que as inundações podem causar danos significativos às cidades e propriedades localizadas às suas margens.
> 
> **Link da Notícia:**
> 
> https://www.cnnbrasil.com.br/nacional/defesa-civil-do-rs-alerta-sobre-risco-de-inundacao-do-rio-taquari/
> 
> **Data da Publicação:**
> 
> 10 de maio de 2023

------------
Deseja fazer alguma pergunta sobre os Alertas da Defesa Civil (s/n): s
Digite sua pergunta sobre os Alertas da Defesa Civil: algum alerta para porto alegre?
Resposta: As notícias fornecidas não mencionam nenhum alerta específico para Porto Alegre. No entanto, é importante ficar atento às informações divulgadas pela Defesa Civil do Rio Grande do Sul, que podem emitir alertas e avisos em tempo real sobre riscos e desastres naturais no estado.

Deseja fazer alguma pergunta sobre os Alertas da Defesa Civil (s/n): s
Digite sua pergunta sobre os Alertas da Defesa Civil: algum rio com risco?
Resposta: Sim, de acordo com as notícias fornecidas, há risco de inundação do Rio Taquari devido às fortes chuvas que atingem a região.

Deseja fazer alguma pergunta sobre os Alertas da Defesa Civil (s/n): n

Menu Principal:
1. Últimas Informações sobre as Enchentes no RS
2. Lugares para Doações
3. Ítens Necessários para Doações
4. Informações Urgentes da Defesa Civil
5. Prever Áreas de Risco

ERROR:tornado.access:500 POST /v1beta/models/gemini-1.0-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1131.99ms


InternalServerError: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting